In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-alpha0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.1.0
numpy 1.14.5
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras 2.2.4-tf


### tf2.0中动态图机制，tensor可以直接输出和op，无需session

In [2]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t)
print(t[:, 1:])
# 打印每个向量的第二列
print(t[..., 2])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([3. 6.], shape=(2,), dtype=float32)


### tensor op

In [3]:
print(t + 10)
print(tf.square(t))
# 计算t乘t的转置
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


### tensor与numpy的互相转换

In [4]:
print(t.numpy())
print(np.square(t))
np_t = np.array([[1, 2, 3],[4, 5, 6]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int64)


### Scalars 

In [5]:
t = tf.constant(3.75)
print(t.numpy())
print(t.shape)

3.75
()


### tf.strings使用

In [6]:
t = tf.constant('caffe')
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit='UTF8_CHAR'))
print(tf.strings.unicode_decode(t, 'UTF8'))

tf.Tensor(b'caffe', shape=(), dtype=string)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 102 101], shape=(5,), dtype=int32)


In [7]:
# string array
t = tf.constant(['caffe', 'coffee', '咖啡'])
# 会输出list中每个string的长度
print(tf.strings.length(t, unit='UTF8_CHAR'))
r = tf.strings.unicode_decode(t, "UTF8")
print(r)

tf.Tensor([5 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


### RaggedTensor(不完整tensor)   
tensor内部维度不规整

In [8]:
# 定义一个ragged tensor
r = tf.ragged.constant([[12, 11], [12, 333, 3], [], [4]])
# 索引操作
print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[12, 11], [12, 333, 3], [], [4]]>
tf.Tensor([ 12 333   3], shape=(3,), dtype=int32)
<tf.RaggedTensor [[12, 333, 3]]>


拼接两个ragged tensor

In [11]:
# 纵向拼接
r1 = tf.ragged.constant([[23, 12], [32], [], [44, 1]])
print(tf.concat([r, r1], axis=0))

<tf.RaggedTensor [[12, 11], [12, 333, 3], [], [4], [23, 12], [32], [], [44, 1]]>


In [13]:
# 横向拼接，需要保证两个ragged tensor第一个维度相同
r2 = tf.ragged.constant([[12, 2], [32, 10], [], [32]])
print(tf.concat([r, r2], axis=1))

<tf.RaggedTensor [[12, 11, 12, 2], [12, 333, 3, 32, 10], [], [4, 32]]>


### ragged tensor转换成普通tensor

In [14]:
# tensor空缺处需要补0
print(r.to_tensor())

tf.Tensor(
[[ 12  11   0]
 [ 12 333   3]
 [  0   0   0]
 [  4   0   0]], shape=(4, 3), dtype=int32)


### Variables 

In [15]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
# 转换为tensor
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


对变量进行赋值，只能使用assgin函数

In [19]:
v.assign(2 * v)
print(v.numpy())
v[0, 1].assign(34)
print(v.numpy())
v[1].assign([7., 8., 9.])
print(v.numpy())

[[ 8. 68. 24.]
 [32. 40. 48.]]
[[ 8. 34. 24.]
 [32. 40. 48.]]
[[ 8. 34. 24.]
 [ 7.  8.  9.]]


In [20]:
# 如果使用=进行赋值，会发生异常
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
